In [36]:
import random
import time

import numpy as np
from enum import Enum
from sklearn.ensemble import RandomForestRegressor
import mido

In [37]:
NOTES = ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"]


class Scales(Enum):
    IONIAN = 0
    DORIAN = 2
    PHRYGIAN = 4
    LYDIAN = 5
    MIXOLYDIAN = 7
    AEOLIAN = 9
    LOCRIAN = 11


PATTERNS_C_IONIAN = [
    [[0, 1, 3], [0, 2, 3], [0, 2, 3], [0, 2, 4], [0, 1, 3], [0, 1, 3]],
    [[0, 2, 4], [1, 2, 4], [1, 2, 4], [1, 3, 4], [0, 2, 4], [0, 2, 4]],
    [[1, 3], [0, 1, 3], [0, 1, 3], [0, 2, 3], [1, 3, 4], [1, 3]],
    [[0, 2, 4], [0, 2, 4], [0, 2, 4], [1, 2, 4], [0, 2, 3], [0, 2, 4]],
    [[1, 3, 4], [1, 3, 4], [1, 3], [0, 1, 3], [2, 3, 4], [1, 3, 4]],
    [[0, 2, 3], [0, 2, 3], [0, 2, 4], [0, 2, 4], [0, 1, 3], [0, 2, 3]],
    [[1, 2, 4], [1, 2, 4], [1, 3, 4], [1, 3, 4], [0, 2, 4], [1, 2, 4]],
    [[0, 1, 3], [0, 1, 3], [0, 2, 3], [0, 2, 3], [1, 3], [0, 1, 3]],
    [[0, 2, 4], [0, 2, 4], [1, 2, 4], [1, 2, 4], [0, 2, 4], [0, 2, 4]],
    [[1, 3, 4], [1, 3], [0, 1, 3], [0, 1, 3], [1, 3, 4], [1, 3, 4]],
    [[0, 2, 3], [0, 2, 4], [0, 2, 4], [0, 2, 4], [0, 2, 3], [0, 2, 3]],
    [[1, 2, 4], [1, 3, 4], [1, 3, 4], [1, 3], [1, 2, 4], [1, 2, 4]]]

In [38]:
def get_pattern(key, scale=Scales.IONIAN):
    to_index = (len(NOTES) - NOTES.index(key)) + scale.value
    if to_index >= len(PATTERNS_C_IONIAN):
        to_index = to_index - len(PATTERNS_C_IONIAN)

    return PATTERNS_C_IONIAN[to_index:] + PATTERNS_C_IONIAN[:to_index]

In [39]:
def generate_chord(pattern=None, key="C", scale=Scales.IONIAN, position=0, include_position=False, note_count=3):
    # Generate a random chord
    if pattern is None:
        pattern = get_pattern(key, scale=scale)
    chord = []
    chosen_strings = random.sample(range(6), note_count)
    for i in range(6):
        if i in chosen_strings:
            chord.append(random.choice(pattern[position][i]) + include_position * position)
        else:
            chord.append(-1)
    return chord

In [40]:
def generate_note(pattern=None, key="C", scale=Scales.IONIAN, position=0, include_position=False):
    generate_chord(pattern=pattern, key=key, scale=scale, position=position, include_position=include_position,
                   note_count=1)

In [41]:
get_pattern("C", scale=Scales.DORIAN)

[[[1, 3], [0, 1, 3], [0, 1, 3], [0, 2, 3], [1, 3, 4], [1, 3]],
 [[0, 2, 4], [0, 2, 4], [0, 2, 4], [1, 2, 4], [0, 2, 3], [0, 2, 4]],
 [[1, 3, 4], [1, 3, 4], [1, 3], [0, 1, 3], [2, 3, 4], [1, 3, 4]],
 [[0, 2, 3], [0, 2, 3], [0, 2, 4], [0, 2, 4], [0, 1, 3], [0, 2, 3]],
 [[1, 2, 4], [1, 2, 4], [1, 3, 4], [1, 3, 4], [0, 2, 4], [1, 2, 4]],
 [[0, 1, 3], [0, 1, 3], [0, 2, 3], [0, 2, 3], [1, 3], [0, 1, 3]],
 [[0, 2, 4], [0, 2, 4], [1, 2, 4], [1, 2, 4], [0, 2, 4], [0, 2, 4]],
 [[1, 3, 4], [1, 3], [0, 1, 3], [0, 1, 3], [1, 3, 4], [1, 3, 4]],
 [[0, 2, 3], [0, 2, 4], [0, 2, 4], [0, 2, 4], [0, 2, 3], [0, 2, 3]],
 [[1, 2, 4], [1, 3, 4], [1, 3, 4], [1, 3], [1, 2, 4], [1, 2, 4]],
 [[0, 1, 3], [0, 2, 3], [0, 2, 3], [0, 2, 4], [0, 1, 3], [0, 1, 3]],
 [[0, 2, 4], [1, 2, 4], [1, 2, 4], [1, 3, 4], [0, 2, 4], [0, 2, 4]]]

In [42]:
generate_chord(
    key="D",
    scale=Scales.MIXOLYDIAN,
    position=random.randrange(0, 5),
    include_position=True,
    note_count=random.randrange(2, 5))

[-1, 5, -1, -1, -1, 2]

In [43]:
open_string_values = [40, 45, 50, 55, 59, 64]

def get_fret_positions(note):
    positions = []
    for string_value in open_string_values:
        if note >= string_value:
            if (note-string_value) > 22:
                positions.append(-1)
            else:
                positions.append(note - string_value)
        else:
            positions.append(-1)
    return positions

notelist = []
for i in random.sample(range(40, 64), 24):
    notelist.append([i, get_fret_positions(i)])

notelist

[[50, [10, 5, 0, -1, -1, -1]],
 [53, [13, 8, 3, -1, -1, -1]],
 [48, [8, 3, -1, -1, -1, -1]],
 [55, [15, 10, 5, 0, -1, -1]],
 [52, [12, 7, 2, -1, -1, -1]],
 [41, [1, -1, -1, -1, -1, -1]],
 [45, [5, 0, -1, -1, -1, -1]],
 [61, [21, 16, 11, 6, 2, -1]],
 [63, [-1, 18, 13, 8, 4, -1]],
 [59, [19, 14, 9, 4, 0, -1]],
 [56, [16, 11, 6, 1, -1, -1]],
 [51, [11, 6, 1, -1, -1, -1]],
 [62, [22, 17, 12, 7, 3, -1]],
 [44, [4, -1, -1, -1, -1, -1]],
 [47, [7, 2, -1, -1, -1, -1]],
 [54, [14, 9, 4, -1, -1, -1]],
 [58, [18, 13, 8, 3, -1, -1]],
 [49, [9, 4, -1, -1, -1, -1]],
 [60, [20, 15, 10, 5, 1, -1]],
 [43, [3, -1, -1, -1, -1, -1]],
 [57, [17, 12, 7, 2, -1, -1]],
 [46, [6, 1, -1, -1, -1, -1]],
 [40, [0, -1, -1, -1, -1, -1]],
 [42, [2, -1, -1, -1, -1, -1]]]

In [44]:
mid = mido.MidiFile("twinkle-twinkle-little-star.mid")
mid

MidiFile(type=1, ticks_per_beat=256, tracks=[
  MidiTrack([
    MetaMessage('time_signature', numerator=4, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0),
    MetaMessage('key_signature', key='C', time=0),
    MetaMessage('set_tempo', tempo=631577, time=0),
    MetaMessage('track_name', name='Greensleeves', time=0),
    MetaMessage('text', text='Traditional', time=0),
    MetaMessage('copyright', text='Jim Paterson', time=0),
    MetaMessage('end_of_track', time=1)]),
  MidiTrack([
    Message('program_change', channel=0, program=0, time=0),
    Message('control_change', channel=0, control=121, value=0, time=0),
    Message('control_change', channel=0, control=64, value=0, time=0),
    Message('control_change', channel=0, control=10, value=63, time=0),
    Message('control_change', channel=0, control=7, value=95, time=0),
    Message('note_on', channel=0, note=60, velocity=77, time=0),
    Message('note_on', channel=0, note=52, velocity=63, time=0),
    Mess

In [45]:
data = []
# msg is type mido.MetaMessage
prevTime = 0
for msg in mid:
    if msg.type == "note_on" or msg.type == "note_off":
        calc_time = round(msg.time * (487/1.2014765585) * 1000)/1000
        data.append({
            "activated": int(msg.type == "note_on"),
            "velocity": msg.bytes()[-1],
            "note": msg.bytes()[1],
            "time": calc_time + prevTime})
        prevTime += calc_time
data

[{'activated': 1, 'velocity': 77, 'note': 60, 'time': 0.0},
 {'activated': 1, 'velocity': 63, 'note': 52, 'time': 0.0},
 {'activated': 1, 'velocity': 64, 'note': 48, 'time': 0.0},
 {'activated': 0, 'velocity': 0, 'note': 60, 'time': 244.0},
 {'activated': 1, 'velocity': 83, 'note': 60, 'time': 256.0},
 {'activated': 0, 'velocity': 0, 'note': 52, 'time': 487.0},
 {'activated': 0, 'velocity': 0, 'note': 48, 'time': 487.0},
 {'activated': 0, 'velocity': 0, 'note': 60, 'time': 500.0},
 {'activated': 1, 'velocity': 90, 'note': 67, 'time': 512.0},
 {'activated': 1, 'velocity': 77, 'note': 60, 'time': 512.0},
 {'activated': 1, 'velocity': 71, 'note': 48, 'time': 512.0},
 {'activated': 0, 'velocity': 0, 'note': 67, 'time': 756.0},
 {'activated': 1, 'velocity': 87, 'note': 67, 'time': 768.0},
 {'activated': 0, 'velocity': 0, 'note': 60, 'time': 999.0},
 {'activated': 0, 'velocity': 0, 'note': 48, 'time': 999.0},
 {'activated': 0, 'velocity': 0, 'note': 67, 'time': 1012.0},
 {'activated': 1, 've

In [46]:
def get_notes_at_time(notes_data, at_time):
    # detect which notes are "activated" at a given time
    notes = {}
    for note in notes_data:
        if note["time"] <= at_time:
            if note["activated"]:
                notes[note["note"]] = note

            else:
                if note["note"] in list(notes.keys()):
                    notes.pop(note["note"])
    return list(notes.values())

def get_chord_at_time(note_data, at_time):
    fret_positions = []
    for note in get_notes_at_time(data, at_time):
        fret_positions.append(get_fret_positions(note["note"]))
    strings = []

    chord = np.full(6, -1)

    for position in fret_positions:
        # get the lowest not -1 fret position and index of lowest fret position in each position list
        lowest = min([x for x in position if x != -1 and position.index(x) not in strings])
        strings.append(position.index(lowest))
        chord[position.index(lowest)] = lowest

    return chord

chords = []
for i in range(0, 12263, 84):
    chords.append(get_chord_at_time(data, i))

chords

[array([-1,  3,  2, -1,  1, -1]),
 array([-1,  3,  2, -1,  1, -1]),
 array([-1,  3,  2, -1,  1, -1]),
 array([-1,  3,  2, -1, -1, -1]),
 array([-1,  3,  2, -1,  1, -1]),
 array([-1,  3,  2, -1,  1, -1]),
 array([-1, -1, -1, -1, -1, -1]),
 array([-1,  3, -1, -1,  1,  3]),
 array([-1,  3, -1, -1,  1,  3]),
 array([-1,  3, -1, -1,  1, -1]),
 array([-1,  3, -1, -1,  1,  3]),
 array([-1,  3, -1, -1,  1,  3]),
 array([-1, -1, -1, -1, -1,  3]),
 array([ 1, -1, -1, -1,  1,  5]),
 array([ 1, -1, -1, -1,  1,  5]),
 array([ 1, -1, -1, -1,  1,  5]),
 array([ 1, -1, -1, -1,  1,  5]),
 array([ 1, -1, -1, -1,  1,  5]),
 array([-1, -1, -1, -1, -1,  5]),
 array([-1,  3, -1, -1,  1,  3]),
 array([-1,  3, -1, -1,  1,  3]),
 array([-1,  3, -1, -1,  1,  3]),
 array([-1,  3, -1, -1,  1,  3]),
 array([-1,  3, -1, -1,  1,  3]),
 array([-1,  3, -1, -1,  1,  3]),
 array([ 1, -1, -1,  2, -1,  1]),
 array([ 1, -1, -1,  2, -1,  1]),
 array([ 1, -1, -1,  2, -1,  1]),
 array([ 1, -1, -1,  2, -1,  1]),
 array([ 1, -1

In [47]:
NOTES = ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"]


class Scales(Enum):
    IONIAN = 0
    DORIAN = 2
    PHRYGIAN = 4
    LYDIAN = 5
    MIXOLYDIAN = 7
    AEOLIAN = 9
    LOCRIAN = 11


PATTERNS_C_IONIAN = [
    [[0, 1, 3], [0, 2, 3], [0, 2, 3], [0, 2, 4], [0, 1, 3], [0, 1, 3]],
    [[0, 2, 4], [1, 2, 4], [1, 2, 4], [1, 3, 4], [0, 2, 4], [0, 2, 4]],
    [[1, 3], [0, 1, 3], [0, 1, 3], [0, 2, 3], [1, 3, 4], [1, 3]],
    [[0, 2, 4], [0, 2, 4], [0, 2, 4], [1, 2, 4], [0, 2, 3], [0, 2, 4]],
    [[1, 3, 4], [1, 3, 4], [1, 3], [0, 1, 3], [2, 3, 4], [1, 3, 4]],
    [[0, 2, 3], [0, 2, 3], [0, 2, 4], [0, 2, 4], [0, 1, 3], [0, 2, 3]],
    [[1, 2, 4], [1, 2, 4], [1, 3, 4], [1, 3, 4], [0, 2, 4], [1, 2, 4]],
    [[0, 1, 3], [0, 1, 3], [0, 2, 3], [0, 2, 3], [1, 3], [0, 1, 3]],
    [[0, 2, 4], [0, 2, 4], [1, 2, 4], [1, 2, 4], [0, 2, 4], [0, 2, 4]],
    [[1, 3, 4], [1, 3], [0, 1, 3], [0, 1, 3], [1, 3, 4], [1, 3, 4]],
    [[0, 2, 3], [0, 2, 4], [0, 2, 4], [0, 2, 4], [0, 2, 3], [0, 2, 3]],
    [[1, 2, 4], [1, 3, 4], [1, 3, 4], [1, 3], [1, 2, 4], [1, 2, 4]]]

In [48]:
def get_pattern(key, scale=Scales.IONIAN):
    to_index = (len(NOTES) - NOTES.index(key)) + scale.value
    if to_index >= len(PATTERNS_C_IONIAN):
        to_index = to_index - len(PATTERNS_C_IONIAN)

    return PATTERNS_C_IONIAN[to_index:] + PATTERNS_C_IONIAN[:to_index]

In [49]:
def generate_chord(pattern=None, key="C", scale=Scales.IONIAN, position=0, include_position=False, note_count=3):
    # Generate a random chord
    if pattern is None:
        pattern = get_pattern(key, scale=scale)
    chord = []
    chosen_strings = random.sample(range(6), note_count)
    for i in range(6):
        if i in chosen_strings:
            chord.append(random.choice(pattern[position][i]) + include_position * position)
        else:
            chord.append(-1)
    return chord

In [50]:
def generate_note(pattern=None, key="C", scale=Scales.IONIAN, position=0, include_position=False):
    generate_chord(pattern=pattern, key=key, scale=scale, position=position, include_position=include_position,
                   note_count=1)

In [51]:
get_pattern("C", scale=Scales.DORIAN)

[[[1, 3], [0, 1, 3], [0, 1, 3], [0, 2, 3], [1, 3, 4], [1, 3]],
 [[0, 2, 4], [0, 2, 4], [0, 2, 4], [1, 2, 4], [0, 2, 3], [0, 2, 4]],
 [[1, 3, 4], [1, 3, 4], [1, 3], [0, 1, 3], [2, 3, 4], [1, 3, 4]],
 [[0, 2, 3], [0, 2, 3], [0, 2, 4], [0, 2, 4], [0, 1, 3], [0, 2, 3]],
 [[1, 2, 4], [1, 2, 4], [1, 3, 4], [1, 3, 4], [0, 2, 4], [1, 2, 4]],
 [[0, 1, 3], [0, 1, 3], [0, 2, 3], [0, 2, 3], [1, 3], [0, 1, 3]],
 [[0, 2, 4], [0, 2, 4], [1, 2, 4], [1, 2, 4], [0, 2, 4], [0, 2, 4]],
 [[1, 3, 4], [1, 3], [0, 1, 3], [0, 1, 3], [1, 3, 4], [1, 3, 4]],
 [[0, 2, 3], [0, 2, 4], [0, 2, 4], [0, 2, 4], [0, 2, 3], [0, 2, 3]],
 [[1, 2, 4], [1, 3, 4], [1, 3, 4], [1, 3], [1, 2, 4], [1, 2, 4]],
 [[0, 1, 3], [0, 2, 3], [0, 2, 3], [0, 2, 4], [0, 1, 3], [0, 1, 3]],
 [[0, 2, 4], [1, 2, 4], [1, 2, 4], [1, 3, 4], [0, 2, 4], [0, 2, 4]]]

In [52]:
generate_chord(
    key="D",
    scale=Scales.MIXOLYDIAN,
    position=random.randrange(0, 5),
    include_position=True,
    note_count=random.randrange(2, 5))

[2, 3, -1, -1, 3, -1]

In [53]:
mid = mido.MidiFile("twinkle-twinkle-little-star.mid")
mid

MidiFile(type=1, ticks_per_beat=256, tracks=[
  MidiTrack([
    MetaMessage('time_signature', numerator=4, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0),
    MetaMessage('key_signature', key='C', time=0),
    MetaMessage('set_tempo', tempo=631577, time=0),
    MetaMessage('track_name', name='Greensleeves', time=0),
    MetaMessage('text', text='Traditional', time=0),
    MetaMessage('copyright', text='Jim Paterson', time=0),
    MetaMessage('end_of_track', time=1)]),
  MidiTrack([
    Message('program_change', channel=0, program=0, time=0),
    Message('control_change', channel=0, control=121, value=0, time=0),
    Message('control_change', channel=0, control=64, value=0, time=0),
    Message('control_change', channel=0, control=10, value=63, time=0),
    Message('control_change', channel=0, control=7, value=95, time=0),
    Message('note_on', channel=0, note=60, velocity=77, time=0),
    Message('note_on', channel=0, note=52, velocity=63, time=0),
    Mess

In [54]:
data = []
# msg is type mido.MetaMessage
prevTime = 0
for msg in mid:
    if msg.type == "note_on" or msg.type == "note_off":
        calc_time = round(msg.time * (487/1.2014765585))
        data.append({
            "activated": int(msg.type == "note_on"),
            "velocity": msg.bytes()[-1],
            "note": msg.bytes()[1],
            "time": calc_time + prevTime})
        prevTime += calc_time
data

[{'activated': 1, 'velocity': 77, 'note': 60, 'time': 0},
 {'activated': 1, 'velocity': 63, 'note': 52, 'time': 0},
 {'activated': 1, 'velocity': 64, 'note': 48, 'time': 0},
 {'activated': 0, 'velocity': 0, 'note': 60, 'time': 244},
 {'activated': 1, 'velocity': 83, 'note': 60, 'time': 256},
 {'activated': 0, 'velocity': 0, 'note': 52, 'time': 487},
 {'activated': 0, 'velocity': 0, 'note': 48, 'time': 487},
 {'activated': 0, 'velocity': 0, 'note': 60, 'time': 500},
 {'activated': 1, 'velocity': 90, 'note': 67, 'time': 512},
 {'activated': 1, 'velocity': 77, 'note': 60, 'time': 512},
 {'activated': 1, 'velocity': 71, 'note': 48, 'time': 512},
 {'activated': 0, 'velocity': 0, 'note': 67, 'time': 756},
 {'activated': 1, 'velocity': 87, 'note': 67, 'time': 768},
 {'activated': 0, 'velocity': 0, 'note': 60, 'time': 999},
 {'activated': 0, 'velocity': 0, 'note': 48, 'time': 999},
 {'activated': 0, 'velocity': 0, 'note': 67, 'time': 1012},
 {'activated': 1, 'velocity': 86, 'note': 69, 'time':

In [55]:
open_string_values = [40, 45, 50, 55, 59, 64]

def get_fret_positions(note):
    positions = []
    for string_value in open_string_values:
        if note["note"] >= string_value:
            if (note["note"]-string_value) > 22:
                positions.append(-1)
            else:
                positions.append(note["note"] - string_value)
        else:
            positions.append(-1)
    return positions

In [56]:
def get_notes_at_time(notes_data, at_time):
    # detect which notes are "activated" at a given time
    notes = {}
    for note in notes_data:
        if note["time"] <= at_time:
            if note["activated"]:
                notes[note["note"]] = note
            else:
                if note["note"] in list(notes.keys()):
                    notes.pop(note["note"])
    return list(notes.values())

In [57]:
def get_chord_at_time(note_data, at_time):
    fret_positions_velocity = []
    for note in get_notes_at_time(note_data, at_time):
        fret_positions_velocity.append([get_fret_positions(note), note["velocity"], note["time"], at_time])
    strings = []
    chord = np.full((6, 4), -1)
    for note_i in range(6):
        chord[note_i, -1] = at_time
    for position in fret_positions_velocity:
        # get the lowest not -1 fret position and index of lowest fret position in each position list
        lowest = min([x for x in position[0] if x != -1 and position[0].index(x) not in strings])
        strings.append(position[0].index(lowest))
        chord[position[0].index(lowest)] = [lowest, position[1], position[2], position[3]]

    return chord

get_chord_at_time(data, 600)

array([[ -1,  -1,  -1, 600],
       [  3,  71, 512, 600],
       [ -1,  -1,  -1, 600],
       [ -1,  -1,  -1, 600],
       [  1,  77, 512, 600],
       [  3,  90, 512, 600]])

In [58]:
def get_keyframes(note_data):
    keyframes = set()
    for note in note_data:
        keyframes.add(note["time"])

    return list(sorted(keyframes))

#find smallest distance between keyframes
def get_smallest_distance(keyframes):
    smallest = 100000
    for key_i in range(len(keyframes)-1):
        if keyframes[key_i+1] - keyframes[key_i] < smallest:
            smallest = keyframes[key_i+1] - keyframes[key_i]
    return smallest

chords = np.full((len(get_keyframes(data)), 6, 4), -1)
for k_i, keyframe in enumerate(get_keyframes(data)):
    chords[k_i] = get_chord_at_time(data, keyframe)

chords

array([[[   -1,    -1,    -1,     0],
        [    3,    64,     0,     0],
        [    2,    63,     0,     0],
        [   -1,    -1,    -1,     0],
        [    1,    77,     0,     0],
        [   -1,    -1,    -1,     0]],

       [[   -1,    -1,    -1,   244],
        [    3,    64,     0,   244],
        [    2,    63,     0,   244],
        [   -1,    -1,    -1,   244],
        [   -1,    -1,    -1,   244],
        [   -1,    -1,    -1,   244]],

       [[   -1,    -1,    -1,   256],
        [    3,    64,     0,   256],
        [    2,    63,     0,   256],
        [   -1,    -1,    -1,   256],
        [    1,    83,   256,   256],
        [   -1,    -1,    -1,   256]],

       ...,

       [[   -1,    -1,    -1, 11764],
        [   -1,    -1,    -1, 11764],
        [   -1,    -1,    -1, 11764],
        [   -1,    -1,    -1, 11764],
        [   -1,    -1,    -1, 11764],
        [   -1,    -1,    -1, 11764]],

       [[   -1,    -1,    -1, 11776],
        [    3,    74, 11776,

In [30]:
# Save 3D chord array to CSV
np.savetxt("twinkle.csv", chords.reshape((chords.shape[0], -1)), delimiter=",", fmt="%d")

# Example C# code for loading CSV and reshaping to 3D array
# int[,] chords = new int[File.ReadLines("twinkle.csv").Count(), 6, 4];
# int[] chords1D = File.ReadAllText("twinkle.csv").Split(',').Select(int.Parse).ToArray();
# for (int i = 0; i < chords1D.Length; i++)
# {
#     chords[i / 24, (i / 4) % 6, i % 4] = chords1D[i];
# }




In [26]:
def post_process_duration(y_pred):
    # Round duration to the nearest multiple of 0.125
    y_pred[:] = np.round(y_pred[:] / 0.125) * 0.125

    return y_pred

In [27]:
def post_process_velocity(y_pred):
    # Clip velocity to the range 1-127
    y_pred[:] = np.clip(np.round(y_pred[:]), 1, 127)

    return y_pred

In [ ]:
def preprocess_input(x):
    # Convert chords to binary
    x = np.array([[''.join([bin(x)[2:].zfill(6) for x in row]) for row in level] for level in (x + 1)])

    return x

In [ ]:
POPULATION = 100
NOTES_PER = 100

In [ ]:
rand_chords_x = np.array([[generate_chord(position=i % 12, note_count=(random.randrange(1, 5)), include_position=True) for i in range(NOTES_PER)] for _ in range(POPULATION)])
rand_chords_x

In [ ]:
rand_chords_x_flat = preprocess_input(rand_chords_x)
rand_chords_x_flat

In [ ]:
rand_durations = np.array(
    [[round(random.random() / 0.125) * 0.125 for _ in range(NOTES_PER)] for _ in range(POPULATION)])
rand_durations

In [ ]:
rand_velocities = np.array([[random.randrange(50, 127) for _ in range(NOTES_PER)] for _ in range(POPULATION)])

rand_velocities

In [ ]:
dur_regr = RandomForestRegressor(n_estimators=100)
dur_regr.fit(rand_chords_x_flat, rand_durations)
vel_regr = RandomForestRegressor(n_estimators=100)
vel_regr.fit(rand_chords_x_flat, rand_velocities)

X_test = np.array([[generate_chord(position=i % 12, note_count=(random.randrange(1, 5)), include_position=True) for i in
                    range(NOTES_PER)] for _ in range(POPULATION)])
preprocess_input(X_test)

In [ ]:
post_process_duration(dur_regr.predict(preprocess_input(X_test)))

In [ ]:
post_process_velocity(vel_regr.predict(preprocess_input(X_test)))

In [8]:
def post_process_duration(y_pred):
    # Round duration to the nearest multiple of 0.125
    y_pred[:] = np.round(y_pred[:] / 0.125) * 0.125

    return y_pred

In [9]:
def post_process_velocity(y_pred):
    # Clip velocity to the range 1-127
    y_pred[:] = np.clip(np.round(y_pred[:]), 1, 127)

    return y_pred

In [220]:
def preprocess_input(x):
    # Convert chords to binary
    x = np.array([[''.join([bin(x)[2:].zfill(6) for x in row]) for row in level] for level in (x + 1)])

    return x

In [11]:
POPULATION = 100
NOTES_PER = 100

In [12]:
rand_chords_x = np.array([[generate_chord(position=i % 12, note_count=(random.randrange(1, 5)), include_position=True) for i in range(NOTES_PER)] for _ in range(POPULATION)])
rand_chords_x

array([[[ 3,  0,  0, -1, -1,  0],
        [-1, -1,  3, -1,  5, -1],
        [ 3,  2, -1, -1,  6, -1],
        ...,
        [-1,  5, -1, -1,  3,  1],
        [-1,  2,  3,  4, -1,  3],
        [ 5, -1,  3, -1,  6, -1]],

       [[ 0, -1, -1, -1, -1, -1],
        [-1, -1, -1, -1,  3, -1],
        [-1,  5,  5, -1,  3,  5],
        ...,
        [-1,  5, -1, -1, -1, -1],
        [ 5, -1, -1, -1, -1, -1],
        [-1,  7,  5,  4, -1,  5]],

       [[ 3, -1,  2, -1,  3, -1],
        [-1,  5, -1,  5, -1, -1],
        [ 3, -1, -1,  5,  5, -1],
        ...,
        [ 5, -1, -1,  4,  3, -1],
        [ 3,  3,  2, -1, -1,  5],
        [ 3,  7, -1, -1, -1, -1]],

       ...,

       [[-1,  2, -1,  4, -1,  1],
        [-1,  5, -1, -1, -1, -1],
        [-1,  2,  3, -1, -1, -1],
        ...,
        [-1, -1,  5, -1,  5,  5],
        [ 3, -1, -1,  4, -1, -1],
        [ 3,  3, -1,  5,  6, -1]],

       [[-1, -1,  3, -1, -1, -1],
        [-1, -1,  5, -1,  3, -1],
        [ 5, -1, -1,  4,  5, -1],
        .

In [13]:
rand_chords_x_flat = preprocess_input(rand_chords_x)
rand_chords_x_flat

array([['000100000001000001000000000000000001',
        '000000000000000100000000000110000000',
        '000100000011000000000000000111000000', ...,
        '000000000110000000000000000100000010',
        '000000000011000100000101000000000100',
        '000110000000000100000000000111000000'],
       ['000001000000000000000000000000000000',
        '000000000000000000000000000100000000',
        '000000000110000110000000000100000110', ...,
        '000000000110000000000000000000000000',
        '000110000000000000000000000000000000',
        '000000001000000110000101000000000110'],
       ['000100000000000011000000000100000000',
        '000000000110000000000110000000000000',
        '000100000000000000000110000110000000', ...,
        '000110000000000000000101000100000000',
        '000100000100000011000000000000000110',
        '000100001000000000000000000000000000'],
       ...,
       ['000000000011000000000101000000000010',
        '000000000110000000000000000000000000',
        '0

In [14]:
rand_durations = np.array(
    [[round(random.random() / 0.125) * 0.125 for _ in range(NOTES_PER)] for _ in range(POPULATION)])
rand_durations

array([[0.75 , 0.125, 0.75 , ..., 0.375, 0.375, 0.875],
       [1.   , 0.875, 0.25 , ..., 0.375, 0.125, 0.625],
       [0.625, 0.375, 0.25 , ..., 0.5  , 0.375, 0.625],
       ...,
       [0.5  , 0.375, 0.75 , ..., 0.75 , 0.   , 0.625],
       [0.875, 0.25 , 0.5  , ..., 0.   , 0.5  , 0.125],
       [0.625, 1.   , 0.125, ..., 0.625, 0.5  , 0.25 ]])

In [15]:
rand_velocities = np.array([[random.randrange(50, 127) for _ in range(NOTES_PER)] for _ in range(POPULATION)])

rand_velocities

array([[ 77,  60,  80, ..., 104, 120,  75],
       [120,  81,  92, ..., 117,  94,  70],
       [117, 102,  65, ..., 118, 119, 122],
       ...,
       [ 83, 106,  94, ..., 120,  56,  94],
       [109, 118,  96, ..., 100,  78,  72],
       [110, 113,  72, ...,  74, 111,  52]])

In [16]:
dur_regr = RandomForestRegressor(n_estimators=100)
dur_regr.fit(rand_chords_x_flat, rand_durations)
vel_regr = RandomForestRegressor(n_estimators=100)
vel_regr.fit(rand_chords_x_flat, rand_velocities)

X_test = np.array([[generate_chord(position=i % 12, note_count=(random.randrange(1, 5)), include_position=True) for i in
                    range(NOTES_PER)] for _ in range(POPULATION)])
preprocess_input(X_test)

array([['000000000000000011000101000100000100',
        '000000000011000011000011000110000000',
        '000100000000000011000101000000000110', ...,
        '000010000000000110000000000110000110',
        '000100000000000000000000000110000000',
        '001000001000000000000101000100000000'],
       ['000000000000000011000000000001000000',
        '000000000000000000000101000000000000',
        '000000000000000110000000000000000000', ...,
        '000000000000000000000011000010000000',
        '000000000000000100000101000000000100',
        '000000000110001000000000000000000000'],
       ['000010000011000011000000000000000000',
        '000010000000000100000011000000000010',
        '000100000011000000000011000000000100', ...,
        '000000000000000000000000000100000000',
        '000000000000000011000011000000000000',
        '000000000000000000000000000110000000'],
       ...,
       ['000001000000000000000000000000000100',
        '000000000110000000000000000000000000',
        '0

In [17]:
post_process_duration(dur_regr.predict(preprocess_input(X_test)))

array([[0.5  , 0.5  , 0.5  , ..., 0.5  , 0.5  , 0.625],
       [0.625, 0.5  , 0.5  , ..., 0.5  , 0.5  , 0.625],
       [0.5  , 0.5  , 0.5  , ..., 0.5  , 0.5  , 0.625],
       ...,
       [0.5  , 0.5  , 0.5  , ..., 0.5  , 0.5  , 0.5  ],
       [0.625, 0.5  , 0.625, ..., 0.5  , 0.5  , 0.5  ],
       [0.5  , 0.375, 0.5  , ..., 0.625, 0.5  , 0.625]])

In [18]:
post_process_velocity(vel_regr.predict(preprocess_input(X_test)))

array([[89., 88., 81., ..., 86., 86., 85.],
       [87., 93., 86., ..., 87., 92., 90.],
       [84., 95., 85., ..., 86., 92., 89.],
       ...,
       [85., 99., 88., ..., 92., 95., 91.],
       [83., 94., 92., ..., 86., 88., 86.],
       [83., 89., 87., ..., 85., 92., 86.]])